# README.md
#get T recovery status for the 167 pts localized disease cohort using the 50ng/dL threshold 

## Table of Contents
* [Read data, clean up and get stats](#first-bullet)
* [Get the baseline T dates and values](#second-bullet)
* [Get the T recovery status](#third-bullet)

In [7]:
import numpy as np
import pandas as pd
import sys, getopt
import csv
from datetime import datetime
import itertools

from matplotlib import pyplot
from pylab import *
import matplotlib.pyplot as plt
%matplotlib inline  
import seaborn as sns

# Read data, clean up and get stats <a class="anchor" id="first-bullet"></a>

In [8]:
df1=pd.read_excel("./data/T_recovery_localized_disease_167pts_cohort.xlsx", header=0)
df1.fillna('', inplace=True)
#df1['inital_ADT_end_date'] = pd.to_datetime(df1['inital_ADT_end_date'])
list_pt=df1["person_id"].to_list()
print(shape(df1))

(167, 14)


In [9]:
df1[:3]

,person_id,status_manual_ok,status_tbaseline,status_rec,ADT_start,ADT_end,baseline_tdate,baseline_tvalue,rec_tdate,rec_tvalue,person_id.1,Salvage RT+ADT (Y/N),evidence for E,ADT used as part of localized disease (Y/N)
0,38,Y,Y,Y,2015-11-18,2016-07-18,2015-11-18,288.37,2018-04-16 00:00:00,317.16,38,N,"Given his age and high risk disease, he is not...",Y
1,81,Y,Y,Y,2014-04-15,2016-06-09,2014-04-10,313.07,2017-04-03 00:00:00,309.59,81,Y,,Y
2,119,Y,Y,Y,2013-07-02,2013-10-02,2013-06-17,403.10,2015-09-21 00:00:00,331.74,119,N,,Y


In [13]:
df_tvalue=pd.read_excel("./data/pca_tresults_20210119_BP_fixed.xlsx", header=0)
#df3=pd.read_csv("just_start_empty.csv", header=0)
df_tvalue.replace(r'', np.NaN)
df_tvalue[:3]

df_tvalue_grouped=df_tvalue.groupby(["person_id"]).size().reset_index(name='counts')
list_pt_t_test=df_tvalue_grouped["person_id"].to_list()
print("list of pt ids with t test results")
print(len(list_pt_t_test))

print("overlapping with t test")
overlap_list1=[x for x in list_pt if x in list_pt_t_test]
print(len(overlap_list1))

list of pt ids with t test results
4490
overlapping with t test
167


In [14]:
def closest(lst, K): 
      return lst[min(range(len(lst)), key = lambda i: abs(lst[i] - K))] 

In [17]:
df1[:3]

,person_id,status_manual_ok,status_tbaseline,status_rec,ADT_start,ADT_end,baseline_tdate,baseline_tvalue,rec_tdate,rec_tvalue,person_id.1,Salvage RT+ADT (Y/N),evidence for E,ADT used as part of localized disease (Y/N)
0,38,Y,Y,Y,2015-11-18,2016-07-18,2015-11-18,288.37,2018-04-16 00:00:00,317.16,38,N,"Given his age and high risk disease, he is not...",Y
1,81,Y,Y,Y,2014-04-15,2016-06-09,2014-04-10,313.07,2017-04-03 00:00:00,309.59,81,Y,,Y
2,119,Y,Y,Y,2013-07-02,2013-10-02,2013-06-17,403.10,2015-09-21 00:00:00,331.74,119,N,,Y


In [18]:
list_pt[:3]

[38, 81, 119]

In [19]:
threshold=50

# T recovery status

In [22]:
result_status_list_rec=[]
df2=df_tvalue

for y in list_pt:    
    df1_pt=df1[(df1["person_id"] == y)]
    df1_pt['ADT_end'] = pd.to_datetime(df1_pt['ADT_end'])
    
    if(len(df1_pt.ADT_end) == 0):        
        b='NaN'
        #gap='NaN'
        tdate='NaN'
        tvalue='NaN'
        status="NaN"
        N_t_dates='NaN'
        N_t_dates_afterADT='NaN'

    if(len(df1_pt.ADT_end)!= 0):        
        b=df1_pt['ADT_end'].to_list()[0]
        
        df2_pt=df2[(df2["person_id"] == y)]    
        each_pt_tdates = df2_pt['result_date'].to_list()

        ##get rid of nan
        cleaned_a=[]
        cleaned_a= [x for x in each_pt_tdates if str(x) != 'NaN']
        cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']
        cleaned_aaa= [x for x in cleaned_aa if str(x) != 'NaT']
        N_t_dates=len(cleaned_aaa)

        #only keep the ones after ADT end date including on the day
        cleaned_aaaa= [x for x in cleaned_aaa if x >=b]
        start_sorted_bin=sorted(cleaned_aaaa)
        N_t_dates_afterADT=len(cleaned_aaaa)
   
        if(len(start_sorted_bin)==0):
            tdate='NaN' 
            tvalue="NaN"
            status="NaN"

#get the T recovery status

        if(len(start_sorted_bin)>0):
            #recovery subset
            df3_pt=df2_pt[df2_pt.test_results >= threshold][df2_pt.result_date >=b]
            #nonrecovery subset
            df4_pt=df2_pt[df2_pt.test_results < threshold][df2_pt.result_date >=b]

            if not df3_pt.empty:
                status ="Y"
                tdate=df3_pt['result_date'].to_list()[0]
                tvalue=df3_pt['test_results'].to_list()[0]
            
            #if not df4_pt.empty:
            if df3_pt.empty:
                status ="N"
                tdate=df4_pt['result_date'].to_list()[-1]
                tvalue=df4_pt['test_results'].to_list()[-1]
    
    print(y,b,tdate,tvalue,status,N_t_dates,N_t_dates_afterADT)
    result_status_per_pt=[y,b,tdate, tvalue, status,N_t_dates,N_t_dates_afterADT]
    result_status_list_rec.append(result_status_per_pt)
#result_status_list_rec=[]

/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


38 2016-07-18 00:00:00 2017-04-13 00:00:00 109.63 Y 20 13
81 2016-06-09 00:00:00 2016-12-29 00:00:00 228.03 Y 12 10
119 2013-10-02 00:00:00 2014-06-04 00:00:00 185.79 Y 12 10
870 2017-10-13 00:00:00 2018-04-10 00:00:00 228.0 Y 10 7
1257 2015-07-08 00:00:00 2016-01-27 00:00:00 203.25 Y 14 13
1503 2018-11-26 00:00:00 2020-01-10 00:00:00 208.87 Y 6 4
1508 2017-04-16 00:00:00 2017-10-21 00:00:00 54.0 Y 9 5
1583 2013-09-11 00:00:00 2013-10-16 00:00:00 10.0 N 6 1
1704 2015-06-05 00:00:00 2015-12-11 00:00:00 125.0 Y 13 12
2061 2019-02-06 00:00:00 2019-10-09 00:00:00 39.2 N 7 2
2063 2018-03-20 00:00:00 2018-05-17 00:00:00 75.75 Y 11 10
2200 2014-11-03 00:00:00 2014-12-03 00:00:00 26.17 N 4 1
2623 2019-10-03 00:00:00 2020-05-28 00:00:00 177.1 Y 15 1
2652 2019-06-01 00:00:00 2019-12-16 00:00:00 379.41 Y 6 3
2655 2017-11-07 00:00:00 2018-05-10 00:00:00 89.58 Y 15 9
2660 2018-10-09 00:00:00 2019-07-09 00:00:00 633.0 Y 8 6
3030 2015-06-05 00:00:00 2020-03-18 00:00:00 116.55 Y 9 7
3579 2015-03-25 00

36508 2016-07-11 00:00:00 2017-02-13 00:00:00 264.72 Y 18 15
167118 2020-02-19 00:00:00 NaN NaN NaN 2 0
167358 2019-04-24 00:00:00 2019-10-09 00:00:00 109.11 Y 8 5
180346 2019-09-18 00:00:00 2019-10-09 00:00:00 14.55 N 3 2
180780 2020-01-15 00:00:00 2020-05-13 00:00:00 10.46 N 4 2
185086 2020-02-07 00:00:00 2020-06-10 00:00:00 41.84 N 7 3
186766 2020-02-03 00:00:00 2020-09-02 00:00:00 730.6 Y 10 4
187363 2019-06-19 00:00:00 2020-02-24 00:00:00 366.97 Y 9 4


In [34]:
df_result_status_rec=pd.DataFrame(result_status_list_rec)
df_result_status_rec.columns=(['person_id', 'initial_ADT_end_date', 'Tdate_rec_norec', 'Tvalue_rec_norec','Recovery_Status',"N_T_dates","N_T_dates_afterADT"])
df_result_status_rec[:3]

,person_id,initial_ADT_end_date,Tdate_rec_norec,Tvalue_rec_norec,Recovery_Status,N_T_dates,N_T_dates_afterADT
0,38,2016-07-18,2017-04-13,109.63,Y,20,13
1,81,2016-06-09,2016-12-29,228.03,Y,12,10
2,119,2013-10-02,2014-06-04,185.79,Y,12,10


In [48]:
df_result_status_rec[:3]

,person_id,initial_ADT_end_date,Tdate_rec_norec,Tvalue_rec_norec,Recovery_Status,N_T_dates,N_T_dates_afterADT
0,38,2016-07-18,2017-04-13,109.63,Y,20,13
1,81,2016-06-09,2016-12-29,228.03,Y,12,10
2,119,2013-10-02,2014-06-04,185.79,Y,12,10


In [47]:
#crosscheck results: are the subset with T value > 50 all have Status as Y
df_rec_group=df_result_status_rec[df_result_status_rec.Recovery_Status=="Y"]
df_rec_group[:3]

,person_id,initial_ADT_end_date,Tdate_rec_norec,Tvalue_rec_norec,Recovery_Status,N_T_dates,N_T_dates_afterADT
0,38,2016-07-18,2017-04-13,109.63,Y,20,13
1,81,2016-06-09,2016-12-29,228.03,Y,12,10
2,119,2013-10-02,2014-06-04,185.79,Y,12,10


In [54]:
df_result_status_rec.to_csv("./data/167pt_salvage_recovery_status_threshold_"+str(threshold)+"_April6.csv", encoding='utf-8', index=False)

In [53]:
df_rec_pt=df_result_status_rec[df_result_status_rec.Recovery_Status=="Y"]
df_nonrec_pt=df_result_status_rec[df_result_status_rec.Recovery_Status=="N"]

list_pt_rec=df_rec_pt["person_id"].to_list()
list_pt_nonrec=df_nonrec_pt["person_id"].to_list()
#
print(len(list_pt_rec))
print(len(list_pt_nonrec))

121
40
